In [16]:
import numpy as np
import pandas as pd
import datasets
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from torch import nn, Tensor
from tqdm.notebook import tqdm
import datasets
from scipy.optimize import linear_sum_assignment
try:
    import composer.functional as cf
except:
    !pip install mosaicml
    import composer.functional as cf
try:
    import transformers
except:
    !pip install -U adapter-transformers  > /dev/null
    import transformers
try:
    import pytorch_lightning as pl
except:
    !pip install pytorch-lightning
    import pytorch_lightning as pl
try:
    import adabelief_pytorch
except:
    !pip install adabelief_pytorch==0.2.0
    import adabelief_pytorch
from transformers import BertTokenizer, DistilBertTokenizerFast, BertForSequenceClassification
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.9/565.9 kB 10.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.11.4
    Uninstalling torchmetrics-0.11.4:
      Successfully uninstalled torchmetrics-0.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
dask-cuda 23.4.0 requires dask==2023.3.2, but you have dask 2023.5.0 which is inc

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [17]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("research multilable")
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
def prepare_data():
    df = pd.read_json('/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', lines=True)
    df = df.drop(columns=['id', 'submitter', 'authors', 'comments', 'journal-ref', 'doi', 'report-no', 'license', 'versions', 'update_date', 'authors_parsed'])
    df['categories'] = df['categories'].apply(lambda i: i.split(' '))
    category_counts = df.explode('categories')['categories'].value_counts()
    print('Start category clearing... ')

    def clear_categories(category_counts, threshold=20000):
        remain_categories = category_counts > threshold
        def wrapped(category_list):
            temp = remain_categories[category_list]
            return list(temp[temp == True].index)
        return wrapped

    df['categories'] = df['categories'].apply(clear_categories(category_counts))
    print('Finished')
    df['category_number'] = df['categories'].apply(len)
    df = df[df['category_number'] != 0]
    category_counts = df.explode('categories')['categories'].value_counts()
    print(category_counts)
    print(len(df))

    from sklearn.preprocessing import MultiLabelBinarizer
    df['text'] = df['title'] + '. ' + df['abstract']
    df = df.drop(columns=['title', 'abstract'])

    def preprocessing(data):
        data['text'] = [a.strip() for a in data['text']]
        data['text'] = data['text'].str.replace('\n', ' ', regex=False).str.replace('\t', ' ', regex=False).str.replace(r'\s\s+', ' ', regex = True)
        return data

    df = preprocessing(df)

    # Extract the categories column as a list of lists
    categories = []
    for el in df["categories"]:
        categories.extend(el)
    categories = np.unique(categories)
    NUM_LABELS = len(categories)

    # Initialize the MultiLabelBinarizer and fit_transform the categories
    mlb = MultiLabelBinarizer()
    df['labels'] =  mlb.fit_transform(df["categories"].values).tolist()

    df_train = df.sample(frac=0.8, random_state=42)
    df_test = df.drop(df_train.index)
    del(df)

    def there_is_category(category):
        def wrapped(category_list):
            for cat in category_list:
                if cat == category:
                    return True
            return False
        return wrapped

    def normalize_categories(df, num_sample=20000):
        category_count = df.explode('categories')['categories'].value_counts()
        df['taken'] = False
        df['there_is_category'] = False
        categorized_dfs = []
        with tqdm(total=len(category_count)) as pbar:
            for category in reversed(list(category_count.index)):
                pbar.update()
                df['there_is_category'] = df['categories'].apply(there_is_category(category))
                suit_df = df[(df['there_is_category'] == True) & (df['taken'] == False)]
                if len(suit_df) >= num_sample:
                    df_category = suit_df.sample(n=num_sample)
                elif len(suit_df) >  0:
                    # this algorith takes whole remainig rows at least once(!)
                    temp_df_list = []
                    temp_df_list.append(suit_df)
                    temp_df_list.append(suit_df.sample(n=num_sample-len(suit_df), replace=True))
                    df_category = pd.concat(temp_df_list)
                else:
                    df_category = None
                if df_category is not None:
                    df.loc[df['there_is_category'], 'taken'] = True
                    categorized_dfs.append(df_category)
        processed_df = pd.concat(categorized_dfs)
        return processed_df

    df_train = normalize_categories(df_train, int(20000*0.8))
    df_test = normalize_categories(df_test, int(20000*0.2))

    MODEL_PATH = 'allenai/scibert_scivocab_uncased'

    train_dataset = datasets.Dataset.from_pandas(df_train)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, lowercase=True)

    def encode_batch(batch):
        return tokenizer(
          batch["text"],
          max_length=500,
          truncation=True,
          padding="max_length"
        )

    train_dataset = train_dataset.map(encode_batch, batched=True)
    train_dataset = train_dataset.remove_columns(['text'])
    print(len(train_dataset))

    train_dataset = train_dataset.remove_columns(['category_number'])
    train_dataset = train_dataset.remove_columns(['categories'])
    train_dataset = train_dataset.remove_columns(['taken'])
    train_dataset = train_dataset.remove_columns(['there_is_category'])

    import os
    os.mkdir('/kaggle/working/train_dataset')
    train_dataset.save_to_disk('/kaggle/working/train_dataset')

    MODEL_PATH = 'allenai/scibert_scivocab_uncased'

    test_dataset = datasets.Dataset.from_pandas(df_test)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, lowercase=True)

    def encode_batch(batch):
        return tokenizer(
          batch["text"],
          max_length=500,
          truncation=True,
          padding="max_length"
        )

    test_dataset = test_dataset.map(encode_batch, batched=True)

    test_dataset = test_dataset.remove_columns(['text'])
    test_dataset = test_dataset.remove_columns(['category_number'])
    test_dataset = test_dataset.remove_columns(['categories'])
    test_dataset = test_dataset.remove_columns(['taken'])
    test_dataset = test_dataset.remove_columns(['there_is_category'])

    os.mkdir('/kaggle/working/test_dataset')
    print(len(test_dataset))
    test_dataset.save_to_disk('/kaggle/working/test_dataset')

    import shutil
    shutil.make_archive('/kaggle/working/train', 'zip', '/kaggle/working/train_dataset')
    shutil.make_archive('/kaggle/working/test', 'zip', '/kaggle/working/test_dataset')

In [19]:
train_dataset = datasets.load_from_disk('/kaggle/input/arxiv-multilable-classification/train')
test_dataset = datasets.load_from_disk('/kaggle/input/arxiv-multilable-classification/test')

test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [20]:
def scibert_fix(fix_params=True, unfixed_transformers=1):
    def wrapped_fixator(model):
        model.classifier.bias = torch.nn.Parameter(torch.ones(model.classifier.bias.shape[0])*(-4))
        if fix_params:
            model.requires_grad_(False)
            if unfixed_transformers > 0:
                model.classifier.requires_grad_(True)
                model.dropout.requires_grad_(True)
                model.bert.pooler.requires_grad_(True)
                for i in range(unfixed_transformers):
                    model.bert.encoder.layer[11-i].requires_grad_(True)
    return wrapped_fixator

In [21]:
class WeightedBCE(torch.nn.Module):
    def __init__(self, false_weight=0.2, weight=None):
        super(WeightedBCE, self).__init__()
        self.fw = false_weight
        self.weight = weight

    def forward(self, input, target):
        max_val = (-input).clamp(min=0)
        if self.weight is None:
            loss = self.fw*(input - input * target) + (self.fw*(1-target)+target)*(max_val + ((-max_val).exp() + (-input - max_val).exp()).log())
        else:
            loss = self.fw*(input - input * target) + (self.fw*(1-target)+self.weight*target)*(max_val + ((-max_val).exp() + (-input - max_val).exp()).log())
        return loss.mean()

In [22]:
def hamming_score(c, y, threshold=0.5):
    assert 0 <= threshold <= 1, "threshold should be between 0 and 1"
    p, q = c.shape
    return 1.0 / (p * q) * (((c > threshold).astype(int) - y) != 0).astype(float).sum()

In [23]:
class BertFineTuning(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self._config = config
        self.outputs = []

        if self._config['bert_class'] == transformers.AutoModelForSequenceClassification:
            self.net = self._config['bert_class'].from_pretrained(self._config['model_path'], num_labels=self._config['num_classes'])
        else:
            self.net = self._config['bert_class'](**self._config['net_params'], num_classes=self._config['num_classes'])
        self.configure_loss()
        self.configure_net()

        self.valid_truth = []
        self.valid_preds = []

    def configure_net(self):
        if self._config['fix_bert'] is not None:
            self._config['fix_bert'](self.net)

    def configure_loss(self):
        self.loss = self._config['loss_class'](**self._config['loss_params'])

    def train_dataloader(self):
        train_dl = DataLoader(self._config['train_dataset'],
                                                batch_size=self._config['batch_size'],
                                                shuffle=True,
                                                num_workers=self._config['num_workers'],
                                                pin_memory=True)
        return train_dl

    def val_dataloader(self):
        val_dl = DataLoader(self._config['val_dataset'],
                                                batch_size=self._config['batch_size'],
                                                shuffle=True,
                                                num_workers=self._config['num_workers'],
                                                pin_memory=True)
        return val_dl

    def configure_optimizers(self):
        opt = self._config['opt_class'](self.net.parameters(), **self._config['opt_params'])
        scheduler = {
            'scheduler' : self._config['scheduler_class'](opt, **self._config['scheduler_params']),
            'interval' : 'step' if self._config['step_every_batch'] else 'epoch'
        }
        return {'optimizer':opt, 'lr_scheduler':scheduler}

    def training_step(self, batch, batch_idx):
        input_ids_ = batch["input_ids"]
        labels_ = batch['labels']
        masks_ = batch["attention_mask"]
    
        logits = self.net(input_ids = input_ids_, attention_mask = masks_)
        if self._config['bert_class'] == transformers.AutoModelForSequenceClassification:
            logits = logits[0]
        loss = self.loss(logits, labels_.float())

        self.log('loss', loss)
        return {"loss":loss}

    def validation_step(self, batch, batch_idx):
        input_ids_ = batch["input_ids"]
        labels_ = batch['labels']
        masks_ = batch["attention_mask"]

        logits = self.net(input_ids = input_ids_, attention_mask = masks_)
        if self._config['bert_class'] == transformers.AutoModelForSequenceClassification:
            logits = logits[0]

        logits_cpu = torch.sigmoid(logits)
        logits_cpu = logits_cpu.cpu().detach().numpy()
        labels_cpu = labels_.cpu().numpy()
        self.valid_truth.extend(labels_cpu)
        self.valid_preds.extend(logits_cpu)
        return

    def on_validation_epoch_end(self):
        self.valid_truth = np.vstack(list(b for b in self.valid_truth))
        self.valid_preds = np.vstack(list(b for b in self.valid_preds))
        epoch_tst_micro_roc_auc = roc_auc_score(self.valid_truth, self.valid_preds, average = 'micro')
        self.last_valid_truth = self.valid_truth
        self.last_valid_preds = self.valid_preds
        
        thresholds = np.arange(0., 1., 0.02)
        hamming_scores = [self._config['num_classes']*hamming_score(self.valid_preds, self.valid_truth, threshold=threshold) for threshold in thresholds]
        optim_threshold = np.argmin(hamming_scores)*0.02
        self.valid_preds = (self.valid_preds>optim_threshold).astype(int)
        
        epoch_v_accuracy_score = accuracy_score(self.valid_truth, self.valid_preds)
        epoch_v_micro_f1_score = f1_score(self.valid_truth, self.valid_preds, average='micro')
        epoch_hamming_score = hamming_score(self.valid_truth, self.valid_preds, threshold=0.5)*self._config['num_classes']

        self.valid_truth = []
        self.valid_preds = []
            
        self.log("accuracy", epoch_v_accuracy_score)
        self.log("f1", epoch_v_micro_f1_score)
        self.log("roc_auc", epoch_tst_micro_roc_auc)
        self.log("hamming_score", epoch_hamming_score)

In [24]:
config = {
    'bert_class' : transformers.AutoModelForSequenceClassification,
    'model_path' : "allenai/scibert_scivocab_uncased", # used only if 'bert_class' == transformers.AutoModelForSequenceClassification
    'net_params' : {},
    'num_classes' : 57,
    'fix_bert' : scibert_fix(True, 3),
    'opt_class' : adabelief_pytorch.AdaBelief,
    'opt_params' : {
        'lr' : 2e-5,
        'betas' : (0.9, 0.999),
        'eps' : 1e-16,
        'weight_decay' : 1e-3,
        'weight_decouple' : True,
        'rectify' : True,
        'fixed_decay' : False,
        'amsgrad' : False
    },
    'scheduler_class' : torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
    'scheduler_params' : {
        'eta_min' : 1e-5,
        'T_0' : 2
    },
    'step_every_batch' : True,
    'batch_size' : 16,
    'num_workers' : 2,
    'train_dataset' : train_dataset,
    'val_dataset' : test_dataset,
    'loss_class' : WeightedBCE,
    'loss_params' : {'false_weight' : 0.2}, 
    'hamming_threshold' : 0.5
}

In [ ]:
module = BertFineTuning(config)

In [ ]:
logger = pl.loggers.WandbLogger(project='Research MultilableClassification', name='bert_finetune modified loss, bias_init, deeper finetune')
trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=1,
    logger=logger,
    log_every_n_steps=20,
    limit_train_batches=1500*4*2,
    limit_val_batches=350*4*2,
    precision="16-mixed"
)
trainer.fit(module)
wandb.finish()

In [25]:
wandb.finish()

In [25]:
def process_target(batched_target, q_length):
    bsize, n_classes = batched_target.shape
    #batched_target - (batch_size; Num_classes)
    output_target = torch.zeros(bsize, q_length, n_classes).type_as(batched_target)
    padding = torch.ones(bsize, q_length, 1).type_as(output_target)
    output_target = torch.cat((padding, output_target), dim=2)

    non_zero = torch.nonzero(batched_target)
    prev_batch_index = None
    for batch_index, class_index in non_zero:
        if prev_batch_index != batch_index:
            q_index = 0
            prev_batch_index = batch_index
        else:
            q_index += 1
        output_target[batch_index][q_index][class_index + 1] = 1
        output_target[batch_index][q_index][0] = 0

    return output_target

@torch.no_grad()
def hungarian_matcher(logits, target):
    """
    target - (batch_size, num_classes)
    logits - (batch_size, seq_length, num_classes+1) <- 'NoClass' included
    """
    # processed_target - (batch_size, seq_length, num_classes+1)
    shape_to_unfl = logits.shape[:2]
    predictions = logits.flatten(0, 1).softmax(-1).unflatten(0, shape_to_unfl)
    processed_target = process_target(target, q_length=predictions.shape[1]).float()
    
    # batch_cost - (batch_size, seq_length, seq_length)
    batched_cost = -torch.matmul(predictions, processed_target.transpose(1, 2))
    permutation_list = list(linear_sum_assignment(cost)[1] for cost in batched_cost.cpu())
    batched_permutation = torch.from_numpy(np.vstack([permutation for permutation in permutation_list]))
    
    target_permuted = processed_target.gather(1, batched_permutation.unsqueeze(-1).expand(-1, -1, predictions.size(-1)).to(processed_target.device))
    return target_permuted

In [26]:
class HungarianLoss(torch.nn.Module):
    def __init__(self, weight_for_entropy=None):
        super(HungarianLoss, self).__init__()
        self.cross_entropy = nn.CrossEntropyLoss(weight=weight_for_entropy)
        
    def forward(self, logits, labels):
        target_permuted = hungarian_matcher(logits, labels)
        loss = self.cross_entropy(logits.flatten(0, 1), target_permuted.flatten(0, 1))
        return loss

In [27]:
class TransformerPredictor(torch.nn.Module):
    def __init__(self, d_model=768, nhead=8, dim_feedforward=3072, dropout=0.1, batch_first=True, q_length=30, num_classes=57, num_layers=8, n_linear=2, classifier_init_policy='neg_biased'):
        super(TransformerPredictor, self).__init__()
        self.batch_first = batch_first
        
        DecoderLayer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=batch_first, activation='relu')
        self.decoder = nn.TransformerDecoder(DecoderLayer, num_layers)
        self.querry = nn.Parameter(torch.rand(q_length, d_model))
        
        blocks = []
        for _ in range(n_linear - 1):
            blocks.append(nn.Linear(d_model, d_model))
            blocks.append(nn.ReLU())
        blocks.append(nn.Linear(d_model, num_classes + 1))
        self.linear_class = nn.Sequential(*blocks)
        
        self._classifier_init_policy(classifier_init_policy, num_classes)
        
    def _classifier_init_policy(self, classifier_init_policy, num_classes):
        if classifier_init_policy == 'unbiased':
            pass
        elif classifier_init_policy == 'neg_biased':
            self.linear_class[-1].bias = torch.nn.Parameter(torch.ones(num_classes+1)*(-3))
        else:
            raise ValueError('There is no such classifier_init_policy - {}'.format(classifier_init_policy))
        
    def forward(self, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        if self.batch_first:
            q = self.querry.unsqueeze(0).repeat(memory.shape[0], 1, 1)
        else:
            q = self.querry.unsqueeze(1).repeat(1, memory.shape[1], 1)
        decoded = self.decoder(q, memory, tgt_mask=tgt_mask, memory_mask=memory_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask)
        return self.linear_class(decoded)

In [28]:
class BertSetClassifier(torch.nn.Module):
    def __init__(self, predictor_params={}, bert_fix_policy='top1'):
        super(BertSetClassifier, self).__init__()
        self.bert_model = transformers.AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")
        self._fix_bert(bert_fix_policy)
        
        self.decoder = TransformerPredictor(**predictor_params)
        
    def _fix_bert(self, bert_fix_policy):
        if bert_fix_policy == "complete":
            self.bert_model.requires_grad_(False)
        elif bert_fix_policy == "top1":
            self.bert_model.requires_grad_(False)
            self.bert_model.pooler.requires_grad_(True)
            self.bert_model.encoder.layer[11].requires_grad_(True)
        elif bert_fix_policy == 'unfix':
            pass
        else:
            raise ValueError('There is no such bert_fix_policy - {}'.format(bert_fix_policy))
            
    def forward(self, input_ids, attention_mask=None):
        encoded = self.bert_model(input_ids, attention_mask=attention_mask.float())['last_hidden_state']
        logits = self.decoder(encoded, memory_key_padding_mask=~(attention_mask.bool()))
        return logits

In [29]:
def logits2predictions(logits, num_classes=57):
    # logits - (batch_size, seq_length, num_classes+1)
    pred_classes_seq = np.argmax(logits.numpy(), axis=2)
    B, Q = pred_classes_seq.shape
    preds = np.zeros((B, num_classes+1))
    
    indices = np.arange(B)[:, np.newaxis]
    preds[indices, pred_classes_seq] = 1
    preds = preds[:,1:]
    return preds

In [40]:
class BertDec(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self._config = config
        self.outputs = []
        
        self.net = config['net_class'](**config['net_params'])
        self.configure_loss()

        self.valid_truth = []
        self.valid_preds = []

    def configure_loss(self):
        weight_to_entropy = self._config['label_weight'].requires_grad_(False)
        self.loss = HungarianLoss(weight_to_entropy)
        
    def train_dataloader(self):
        train_dl = DataLoader(self._config['train_dataset'],
                                                batch_size=self._config['batch_size'],
                                                shuffle=True,
                                                num_workers=self._config['num_workers'],
                                                pin_memory=True)
        return train_dl

    def val_dataloader(self):
        val_dl = DataLoader(self._config['val_dataset'],
                                                batch_size=self._config['batch_size'],
                                                num_workers=self._config['num_workers'],
                                                pin_memory=True)
        return val_dl

    def configure_optimizers(self):
        opt = self._config['opt_class'](self.net.parameters(), **self._config['opt_params'])
        scheduler = {
            'scheduler' : self._config['scheduler_class'](opt, **self._config['scheduler_params']),
            'interval' : 'step' if self._config['step_every_batch'] else 'epoch'
        }

        return {'optimizer':opt, 'lr_scheduler':scheduler}

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        labels = batch['labels']
        attention_mask = batch["attention_mask"]
    
        logits = self.net(input_ids=input_ids, attention_mask=attention_mask)        
        loss = self.loss(logits, labels.float())

        self.log('loss', loss)
        return {"loss":loss}

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        labels = batch['labels']
        attention_mask = batch["attention_mask"]

        logits = self.net(input_ids=input_ids, attention_mask=attention_mask)
        preds = logits2predictions(logits.cpu().detach(), num_classes=self._config['num_classes'])

        labels_cpu = labels.cpu().numpy()
        self.valid_truth.extend(labels_cpu)
        self.valid_preds.extend(preds)
        return

    def on_validation_epoch_end(self):
        self.valid_truth = np.vstack(list(b for b in self.valid_truth))
        self.valid_preds = np.vstack(list(b for b in self.valid_preds))
        epoch_tst_micro_roc_auc = roc_auc_score(self.valid_truth, self.valid_preds, average = 'micro')
        self.last_valid_truth = self.valid_truth
        self.last_valid_preds = self.valid_preds
        
        epoch_v_accuracy_score = accuracy_score(self.valid_truth, self.valid_preds)
        epoch_v_micro_f1_score = f1_score(self.valid_truth, self.valid_preds, average='micro')
        epoch_hamming_score = hamming_score(self.valid_truth, self.valid_preds, threshold=0.5)*self._config['num_classes']

        self.valid_truth = []
        self.valid_preds = []
            
        self.log("accuracy", epoch_v_accuracy_score)
        self.log("f1", epoch_v_micro_f1_score)
        self.log("roc_auc", epoch_tst_micro_roc_auc)
        self.log("hamming_score", epoch_hamming_score)

In [76]:
num_labels = 0
dl = DataLoader(train_dataset, num_workers=2, batch_size=512, drop_last=True)
with tqdm(total=len(dl)) as pbar:
    for batch in dl:
        pbar.update()
        num_labels += batch['labels']
num_labels = torch.sum(num_labels, dim=0)
label_weight = (1/num_labels) / (1/num_labels).mean()
label_weight = torch.cat((torch.ones(1)*0.1, label_weight))

  0%|          | 0/1687 [00:00<?, ?it/s]

In [78]:
config_dec = {
    'net_class' : BertSetClassifier,
    'net_params' : {'bert_fix_policy':'top1'},
    'num_classes' : 57,
    'opt_class' : adabelief_pytorch.AdaBelief,
    'opt_params' : {
        'lr' : 2e-5,
        'betas' : (0.9, 0.999),
        'eps' : 1e-16,
        'weight_decay' : 1e-3,
        'weight_decouple' : True,
        'rectify' : True,
        'fixed_decay' : False,
        'amsgrad' : False
    },
    'scheduler_class' : torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
    'scheduler_params' : {
        'eta_min' : 1e-5,
        'T_0' : 2
    },
    'step_every_batch' : True,
    'batch_size' : 32,
    'num_workers' : 2,
    'train_dataset' : train_dataset,
    'val_dataset' : test_dataset,
    'apply_alibi' : True,
    'label_weight' : label_weight
}

In [79]:
module = BertDec(config_dec)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
wandb.finish()

In [80]:
logger = pl.loggers.WandbLogger(project='Research MultilableClassification', name='detr-like')
trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=2,
    logger=logger,
    log_every_n_steps=10,
    limit_train_batches=1500*4*2,
    limit_val_batches=350*4*2,
    precision='16-mixed'
)
trainer.fit(module)
wandb.finish()

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [66]:
print('Total number of trainable parameters : {}'.format(sum(parameter.numel() for parameter in module.net.parameters() if parameter.requires_grad)))
print('Total number of trainable parameters : {}'.format(sum(parameter.numel() for parameter in module.net.decoder.parameters() if parameter.requires_grad)))

Total number of trainable parameters : 186183226
Total number of trainable parameters : 76264762


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [31]:
weight_for_entropy = torch.ones(58, dtype=float).to('cuda')
weight_for_entropy[0] = 0.1

in_idx_debug = []
mask_debug = []

dl = DataLoader(train_dataset, batch_size=8)
net = module.net.to('cuda')
Loss = HungarianLoss(weight_for_entropy=weight_for_entropy)
processed_steps = 0
with torch.no_grad():
    for batch in dl:
        in_idx = batch['input_ids'].to('cuda')
        labels = batch['labels'].to('cuda')
        mask = batch['attention_mask'].to('cuda')
        logits = net(in_idx, mask.bool())
        processed_steps += 1
        try:
            loss = Loss(logits, labels)
        except:
            in_idx_debug.append(in_idx)
            mask_debug.append(mask)
            break

In [32]:
processed_steps

8

In [33]:
in_idx_debug[0].shape

torch.Size([8, 500])

In [28]:
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')

Переделать memory_key_padding_mask - когда float - напрямую добавляется!!!

In [15]:
a = train_dataset[0]['attention_mask']
a = ~a.reshape(1, -1).bool()
key_padding_mask = F._canonical_mask(
            mask=a,
            mask_name="key_padding_mask",
            other_type=F._none_or_dtype(None),
            other_name="attn_mask",
            target_type=float
        )